In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/SL_TRUST/cnn trust/update'

/content/drive/MyDrive/Colab Notebooks/SL_TRUST/cnn trust/update


In [ ]:
from copy_cifar10_with_attack import *

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.preprocessing.image import img_to_array, array_to_img
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score

print(tf.__version__)

2.8.0


## Import the dataset

In [ ]:
# # Cifar10, Cifar 100, FashionMNIST
# fashion_mnist = keras.datasets.fashion_mnist
# (train_images, train_label), (test_images, test_label) = fashion_mnist.load_data()
# train_images = train_images / 255.0 ##fashion_mnist
# test_images= test_images / 255.0 ##fashion_mnist
# # train_images=np.stack([train_images]*3, axis=-1)
# # test_images=np.stack([test_images]*3, axis=-1)
# # train_images = np.asarray([img_to_array(array_to_img(im, scale=False).resize((32,32))) for im in train_images])
# # test_images = np.asarray([img_to_array(array_to_img(im, scale=False).resize((32,32))) for im in test_images])

fashion_mnist = keras.datasets.cifar10
(train_images, train_label), (test_images, test_label) = fashion_mnist.load_data()

train_images.shape, train_label.shape, test_images.shape, test_label.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import sklearn
from sklearn.metrics import auc, plot_precision_recall_curve

model_load = tf.keras.models.load_model('/content/drive/MyDrive/cnn trust/update/parameters/model_cnn_cifar10.h5')
model_load.evaluate(test_images, test_label)

y_pred = model_load.predict(test_images)
m = tf.keras.metrics.AUC(num_thresholds=200)
m.update_state(tf.one_hot(test_label[:,0],depth=10), y_pred)
auroc = m.result().numpy()

y_pred = tf.math.argmax(y_pred,axis=1)
precision, recall, f1, _ = precision_recall_fscore_support(test_label, y_pred,average='weighted')

precision, recall, f1, auroc

313/313 [==============================] - 8s 5ms/step - loss: 0.5219 - accuracy: 0.8527


(0.8576079966186256, 0.8527, 0.8530323732877543, 0.97981775)

Reload model and remove the dense layers

In [ ]:
from keras.models import Model

model2= Model(inputs=model_load.input, outputs=model_load.layers[-8].output)
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 depthwise_conv2d (Depthwise  (None, 32, 32, 96)       960       
 Conv2D)                                                         
                                                                 
 dropout (Dropout)           (None, 32, 32, 96)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        55360 

##convert images to features

In [ ]:
test_feature = model2.predict(test_images)              
train_feature = model2.predict(train_images)
train_feature.shape, test_feature.shape

((50000, 4, 4, 512), (10000, 4, 4, 512))

In [ ]:
test_images.shape

(10000, 32, 32, 3)

Build a new small model for transfer learning

In [ ]:
def intitial_W_x(X):
    """
    Initialize input opinion for caseI.
    
    """
    W_dis = np.zeros(X.shape).astype(np.float32)
    W_base = W_dis + 0.5
    W_x = np.array([X, W_dis, 1 - X, W_base]).astype(np.float32)
    W_x = np.moveaxis(W_x, 0, -1)
    
    return W_x

def get_update_matrix(grads_w, grads_b, rs):
    """
    Update the opinion of weight and bias during training process.
    grads_w - gradient evidence matrix of weight
    grads_b - gradient evidence matrix of bias
    rs - positive evidence plus negative evidence
    
    """
    
    W_w = np.array([grads_w/(rs+2),(rs-grads_w)/(rs+2),np.full(grads_w.shape, 2/(rs+2)),
                    np.full(grads_w.shape, 0.5)]).astype(np.float32)
    W_b = np.array([grads_b/(rs+2), (rs-grads_b)/(rs+2), np.full(grads_b.shape, 2/(rs+2)), 
                    np.full(grads_b.shape, 0.5)]).astype(np.float32)
    W_w = np.moveaxis(W_w, 0, -1)
    
    return W_w, W_b.swapaxes(0,1)

def evidence_collect(y, y_pred):
    """
    Collect positive and negative evidence during the training process.

    """
    r = 0
    s = 0
    r_list = [0]*10
    s_list = [0]*10
    
    for j in range(len(y_pred)):
        for i in range(len(y_pred[0])):
            if i == y[j]:
                if y_pred[j][i] > 0.5:
                    r_list[i]+=1
                    r+=1
                else:
                    s+=1
                    s_list[i]+=1
            else:
                if y_pred[j][i] < 0.1:
                    r_list[i]+=1
                else:
                    s_list[i]+=1
                    
    y_N_op = []
    for i in range(len(r_list)):
        y_N_op.append([r_list[i]/(r_list[i]+s_list[i]+2), 
                       s_list[i]/(r_list[i]+s_list[i]+2), 2/(r_list[i]+s_list[i]+2), 0.5])

    
    return [r/(r+s+2), s/(r+s+2), 2/(r+s+2), 0.5], y_N_op

In [ ]:
def padding(opinion, kernel_size):
    """
    Trust map padding with maximum unblief mass on the padding cell. 
    
    """
    image_size = opinion.shape[1]
    p = int((kernel_size - 1)/2)
    padding_size = int(image_size + 2 * p)
    opinion_pad = np.zeros((int(opinion.shape[0]),padding_size,padding_size,int(opinion.shape[-2]),int(opinion.shape[-1])))
    opinion_pad[:,:,:,:] = np.array([0.0, 0.99, 0.01, 0.5])
    opinion_pad[:,p:p+image_size,p:p+image_size,:,:] = opinion
    return opinion_pad

def multi(W_x, W_y): 
    """
    Multi-sources multiplication Operator 

    """
    W_x = W_x.astype(np.float64) 
    W_y = W_y.astype(np.float64)    
    W_b = W_x[0]*W_y[0]+((1-W_x[3])*W_y[3]*W_x[0]*W_y[2]+W_x[3]*(1-W_y[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_d = W_x[1]+W_y[1]-W_x[1]*W_y[1]
    W_u = W_x[2]*W_y[2]+((1-W_y[3])*W_x[0]*W_y[2]+(1-W_x[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_a = W_x[3]*W_y[3]
    return W_b,W_d,W_u,W_a

In [ ]:
def conv_single_step_trust(W_x, W_w, W_b): 
    """
    Compute the output opinion of one convolutional kernel window.
    inputs:
    W_x - input opinion (batch size, kernel size, kernel size, channel, 4)
    W_w - weight opinion (kernel size, kernel size, channel, filter number, 4)
    W_b - bias opinion (filter number, 4)
    
    """    
    W_x = W_x.astype(np.float64)   
    W_w = W_w.numpy().astype(np.float64) 
    W_b = W_b.numpy().astype(np.float64) 
    
    filter_number = W_b.shape[0]
    batch_size = W_x.shape[0]
    fusion_result = []
    W_x_expand = np.tile(np.expand_dims(W_x, axis=(4)), [1,1,1,1,filter_number,1])
    W_w_expand = np.tile(np.expand_dims(W_w, axis=(0)), [batch_size,1,1,1,1,1])
    W_b_expand = np.tile(np.expand_dims(W_b, axis=(0)), [batch_size,1,1])
    W_wx = multi(np.transpose(W_x_expand),np.transpose(W_w_expand)) # (4, 32, 3, 5, 5, 50)
    fusion_result = avg_fusion(np.asarray(W_wx), np.transpose(W_b_expand))
    
    return np.transpose(fusion_result[0]),np.transpose(fusion_result[1])

def avg_fusion(W_wx, W_b):
    """
    Multi-sources average opinion operator.
    W_wx - opinion of input mutiplied with weight opinion (4, filter number, channel, kernel size, kernel size, batch size)
    W_b - bias opinion (4, filter number, batch size)

    """
    
    W_wx = np.reshape(W_wx, (W_wx.shape[0],W_wx.shape[1], 
                             W_wx.shape[2]*W_wx.shape[3]*W_wx.shape[4], W_wx.shape[5])).astype(np.float64) # (4, 32, 75, 50)
    W_b = W_b.astype(np.float64)
    
    n_filter = W_b.shape[1]
    batch_size = W_wx.shape[-1]
    num_para = W_wx.shape[2]
    b_wx, u_wx, a_wx = W_wx[0], W_wx[2], W_wx[3]
    b_b, u_b, a_b = W_b[0], W_b[2], W_b[3]

    u_combine = np.concatenate((u_wx, np.reshape(u_b,(u_b.shape[0], 1, batch_size))), axis=1) # (32, 76, 50)
    b_combine = np.concatenate((b_wx, np.reshape(b_b,(b_b.shape[0], 1, batch_size))), axis=1) # (32, 76, 50)
    u_combine_recip = (np.zeros(u_combine.shape)+1)/u_combine
    
    numerator = np.sum(b_combine * u_combine_recip, axis = 1) # (32, 50)
    denominator = np.sum(u_combine_recip, axis = (1))
    
    b_fusion = numerator / denominator
    u_fusion = (num_para+1) / denominator
    a_fusion = (np.sum(a_wx, axis=(1)) + a_b) / (num_para+1)
    
    return np.array([b_fusion, 1-b_fusion-u_fusion, u_fusion, a_fusion]).astype(np.float32), (b_fusion + u_fusion * a_fusion).astype(np.float32)

In [ ]:
def conv_forward(A_prev, W_w, W_b):
    """
    Forward convolutional opinion calculation.
    A_prev - feature map from last layer (batch_size, H, W, channel) 
    W_w - opinion of weight (kernel_size, kernel_size, 3, filter_num, 4)
    W_b - opinion of bias (filter_num, 4)

    """
    (batch_size, n_H_prev, n_W_prev, _, _) = A_prev.shape
    ( f , f , _, n_C, _) = W_w.shape
    stride = 1
    pad = 1
    A_prev_pad = padding(A_prev,f)

    n_H = int(( n_H_prev - f + 2 * pad )/ stride) + 1
    n_W = int(( n_W_prev - f + 2 * pad )/ stride) + 1
 
    Z = np.zeros((batch_size, n_H, n_W, n_C, 4)) 
    Z_trust = np.zeros((batch_size, n_H, n_W, n_C))

    for h in range(n_H):                       
        for w in range(n_W):                              
            vert_start = h * stride        
            vert_end = vert_start + f       
            horiz_start = w * stride        
            horiz_end = horiz_start + f     
            a_slice_prev = A_prev_pad[:,vert_start:vert_end,horiz_start:horiz_end,:,:]   
            opinion, trust = conv_single_step_trust(a_slice_prev,W_w,W_b)   
            Z[:,h,w,:,:] = opinion
            Z_trust[:,h,w,:] = trust

    return Z, Z_trust

In [ ]:
class MyLayerCONV(tf.keras.layers.Layer):
    """
    Conv opinion computation layer
    
    """
    def __init__(self):
        super(MyLayerCONV, self).__init__()


    def call(self, input1, input2, input3):
        
        Z, Z_trust = conv_forward(input1, input2, input3)
           
        return Z.astype(np.float32), Z_trust.astype(np.float32)


In [ ]:
class MyLayerMaxTrust(tf.keras.layers.Layer):
    """
    Max-trust fuction layer
    A new pooling layer function based on trustworthiness values instead of feature values. 

    """
    def __init__(self):
        super(MyLayerMaxTrust, self).__init__()


    def call(self, x, opinion, trust):
        """
        x - feature map from last layer (batch, H, W, C)
        opinion - corresponding opinion (batch, H, W, C, 4)
        trust - corresponding trust (batch, H, W, C)

        """
        trust_mul = np.zeros(x.shape)
        image_size = x.shape[1]
        opinion_out = np.zeros((opinion.shape[0],int(opinion.shape[1]/2), int(opinion.shape[2]/2),opinion.shape[3], 4)).astype(np.float32)
        for i in range(trust.shape[0]):
            for k in range(trust.shape[-1]):
                input_max = trust[i,:,:,k].reshape((1,trust.shape[1],trust.shape[2],1))
                _, argmax = tf.nn.max_pool_with_argmax(input = input_max, ksize = [1, 2, 2, 1],
                                                    strides = [1, 2, 2, 1], padding = 'VALID')
                argmax_1d = argmax.numpy().flatten()
                for j in range(len(argmax_1d)):
                    trust_mul[i,int(argmax_1d[j]/image_size),int(argmax_1d[j]%image_size),k] = 1
                    opinion_out[i,int(j/int(image_size/2)), int(j%int(image_size/2)),k,:] = opinion[i,int(argmax_1d[j]/image_size),int(argmax_1d[j]%image_size),k,:] 

        x = x * trust_mul

        return x, opinion_out

In [ ]:
class TrustBlock(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.convopinion1 = MyLayerCONV()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             
            kernel_size=[3, 3],
            padding='same',
            activation=tf.nn.relu   
        )
        self.maxtrust1 = MyLayerMaxTrust()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)

    def call(self, inputs):
        """
        x - 50,32,32,3
        W_x - 50,32,32,3,4
        
        
        """
        X, W_x, W_w1, W_b1 = inputs
 
        opinion1, trust1 = self.convopinion1(W_x, W_w1, W_b1)
        x = self.conv1(X)   
        x, pooling_opinion1 = self.maxtrust1(x, opinion1, trust1)
        x = self.pool1(x)    
        x = self.flatten(x) 
        x = self.dense1(x)    
        output = self.dense2(x)                    
        return output,pooling_opinion1

model_maxtrust = TrustBlock()

In [ ]:
num_epochs = 1
batch_size = 128
kernelsize = 3
filter_num1 = 32
# threshold1 = 20
# threshold2 = 15
optimizer = tf.keras.optimizers.Adam()
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
train_acc = []
train_loss = []
opinion_convlist = []
y_update_wb = []
y_N_opinion = []

In [ ]:
def get_batch(train_data, train_label, batch_size):
    index = np.random.randint(0, train_data.shape[0], batch_size)
    return train_data[index, :], train_label[index]

In [ ]:
# batch training process
num_batches = int(train_images.shape[0] // batch_size * num_epochs)

W_w1 = np.array([[[[[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num1)] for _ in range(512)] 
                  for _ in range(kernelsize)] for _ in range(kernelsize)]).astype(np.float32)

W_b1 = np.array([[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num1)]).astype(np.float32) 

grads_list_w1 = np.zeros((kernelsize,kernelsize,512,filter_num1))
grads_list_b1 = np.zeros((filter_num1,))
start = time.time()

for batch_index in range(num_batches):
    print('# of batch:',batch_index)
    X, y = get_batch(train_images, train_label, batch_size) 
    W_x = intitial_W_x(X)
    with tf.GradientTape() as tape:
        y_pred,opinion_conv = model_maxtrust([X, W_x, W_w1, W_b1])
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)

    grads = tape.gradient(loss, model_maxtrust.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model_maxtrust.variables))
    # sparse_categorical_accuracy.reset_states()
    sparse_categorical_accuracy.update_state(y_true=y, y_pred=y_pred)
    print("train accuracy:",sparse_categorical_accuracy.result().numpy())
#     print('Model update end:',(time.time()-start))
      
# update W_w, W_b
    grads_list_w1 = grads_list_w1 + tf.where(abs(grads[0]) < np.average(abs(grads[0])), 1, 0)  #  shape=(5, 5, 3, 8)
    grads_list_b1 = grads_list_b1 + tf.where(abs(grads[1]) < np.average(abs(grads[1])), 1, 0)  #  shape=(8,
    W_w1, W_b1 = get_update_matrix(grads_list_w1, grads_list_b1, batch_index+1)                               

    y_N_update, y_N_op = evidence_collect(y, y_pred)  
    y_N_opinion.append(y_N_op)
    y_update_wb.append(y_N_update)
   
    print('One batch end:',(time.time()-start))

KeyboardInterrupt: ignored

In [ ]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
 
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

In [ ]:
# model_maxtrust.save_weights('/content/drive/MyDrive/cnn trust/update/parameters/model_CIFAR10_transfer')
# save_variable(W_w1, '/content/drive/MyDrive/cnn trust/update/parameters/CIFAR10_weight_with_MAX_Trust_transfer')
# save_variable(W_b1, '/content/drive/MyDrive/cnn trust/update/parameters/CIFAR10_bias_with_MAX_Trust_transfer')

'/content/drive/MyDrive/cnn trust/update/parameters/CIFAR10_bias_with_MAX_Trust_transfer'

In [ ]:
model_maxtrust.load_weights('/content/drive/MyDrive/cnn trust/update/parameters/model_CIFAR10_transfer')
W_w1 = load_variavle('/content/drive/MyDrive/cnn trust/update/parameters/CIFAR10_weight_with_MAX_Trust_transfer')
W_b1 = load_variavle('/content/drive/MyDrive/cnn trust/update/parameters/CIFAR10_bias_with_MAX_Trust_transfer')

In [ ]:
test_dataset = test_feature[:500]
test_data_label = test_label[:500]

In [ ]:
# test data
test_opinion = intitial_W_x(test_dataset)
y_pred_test,opinion_test_last = model_maxtrust([test_dataset, test_opinion, W_w1, W_b1])
sparse_categorical_accuracy.reset_states()
sparse_categorical_accuracy.update_state(y_true=test_data_label, y_pred=y_pred_test)
print("test accuracy:",sparse_categorical_accuracy.result().numpy())

test accuracy: 0.81


In [ ]:
from skimage import util
class CIFARLoader():
    """
    Dataset loader class
    
    """
    def __init__(self):
        (self.train_data, self.train_label), (self.test_data, self.test_label) = tf.keras.datasets.cifar10.load_data()
       
        self.train_data = self.train_data.astype(np.float32)/ 255.0     # [50000, 32, 32, 3]
        self.test_data = self.test_data.astype(np.float32) / 255.0      # [10000, 32, 32, 3]
        self.train_label = self.train_label.astype(np.int32)    # [50000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]
    def get_batch_damage_label(self, batch_size, percentage):
        index = np.random.randint(0, self.num_train_data, batch_size)
        batch_label = np.copy(self.train_label[index])
        batch_label[:int(batch_size*percentage)] = 0
        return self.train_data[index, :], batch_label
    
    def get_local_noise(self, noise_num=400, batch_size=5000):
        """
        Generate noise_num # of gassaion noise knowing location 
        
        """

        process_image = np.copy(self.test_data[:batch_size])
        W_x = intitial_W_x(process_image)
        uncer_mass = np.zeros(W_x[:,0,0,:,0].shape) + 0.01
        base_rate = np.zeros(W_x[:,0,0,:,0].shape) + 0.5
        image_size = process_image.shape[1]
        index_noise = np.random.randint(0, image_size*image_size, noise_num)
        
        for i in index_noise:
            noise_value = np.random.randn(batch_size,3) * 0.4
            feature_value = process_image[:,int(i/image_size),int(i%image_size),:]
            process_image[:,int(i/image_size),int(i%image_size),:] = feature_value + noise_value
            belief_mass = W_x[:,int(i/image_size),int(i%image_size),:,0] 
            disbelief_mass = 1 - belief_mass - uncer_mass
            W_x[:,int(i/image_size),int(i%image_size),:] = np.moveaxis(np.array([belief_mass,disbelief_mass,uncer_mass,base_rate]), 0, -1)
            
        return tf.clip_by_value(process_image, 0, 1), self.test_label[:batch_size], W_x, index_noise
    
    def get_noise(self, noise_mode):
        process_image = np.copy(self.test_data[:400])
        noise_gs_img = util.random_noise(process_image,mode=noise_mode, var=0.01)
        return noise_gs_img, self.test_label[:400]

    def get_test(self, size):

        return self.test_data[:size], self.test_label[:size]
    
data_loader = CIFARLoader()

In [ ]:
X_test_noise, y_test_label = data_loader.get_noise('gaussian')
test_feature_noise = model2.predict(X_test_noise)

In [ ]:
X_test_noise.shape

(400, 32, 32, 3)

In [ ]:
noise_opinion = intitial_W_x(test_feature_noise)
y_pred_test_noise,opinion_noise = model_maxtrust([test_feature_noise, noise_opinion, W_w1, W_b1])
sparse_categorical_accuracy.reset_states()
sparse_categorical_accuracy.update_state(y_true=y_test_label, y_pred=y_pred_test_noise)
print("test accuracy:",sparse_categorical_accuracy.result().numpy())

test accuracy: 0.095


In [ ]:
# test localvar
X_test_local_noise, y_test_local_noise, case1_opinion, noise_index = data_loader.get_local_noise()
case2_opinion = intitial_W_x_location(X_test_local_noise, noise_index)
case3_opinion = intitial_W_x(X_test_local_noise)
y_pred_test_local_noise,_,_,opinion_noise = model_maxtrust([X_test_local_noise, case3_opinion, W_w1, W_b1, W_w2, W_b2])
sparse_categorical_accuracy.reset_states()
sparse_categorical_accuracy.update_state(y_true=y_test_local_noise, y_pred=y_pred_test_local_noise)
print("test accuracy:",sparse_categorical_accuracy.result().numpy())

In [ ]:
def get_NN_trust(opinion_last_layer, true_label, pre_label, y_update_wb):
    # compute dense opinion
    opinion_dense = np.average(np.array(opinion_last_layer), axis=0)
    opinion_dense = np.reshape(opinion_dense, (int(opinion_dense.shape[0]*opinion_dense.shape[1]*opinion_dense.shape[2]),4))
    y_true_op = [1.0, 0.0, 0.0, 0.5]
    W_y_update = evidence_collect_test(true_label, pre_label)
    
    # compute Backward opinion of neuron W_N_Y  
    W_N_Y=[]
    for j in W_y_update:
        W_N_Y.append(multi(j,y_true_op)) # change when add flaw in label
        
    W_w_dense = y_update_wb[-1]
    W_b_dense = y_update_wb[-1]

    W_xw=[]
    W_xw.append(W_b_dense)
    for j in range(opinion_dense.shape[0]):
        W_xw.append(multi(W_w_dense,opinion_dense[j])) # (513, 4)
    dense_out = fusion(np.array(W_xw))
    #     print('Underflow or not: ',np.isnan(np.min(np.array(dense1_out_list))))

    # last layer
    W_xw=[]
    W_xw.append(W_b_dense)
    for j in range(64):
        W_xw.append(multi(W_w_dense,dense_out))
    W_NN = fusion(np.array(W_xw))
    
    # compute last layer output opinion and trust
    W_XY_one = []
    for j in range(10):
        W_XY_one.append(fusion_2(W_NN,W_N_Y[j]))
    W_NN = fusion(np.array(W_XY_one))
    W_trust = W_NN[0]+W_NN[2]*W_NN[3]
    
    return W_trust, W_NN

In [ ]:
from tensorflow.keras import Sequential, datasets,models,layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, DepthwiseConv2D, BatchNormalization
from tensorflow.keras.models import load_model
tf.random.set_seed(1)
hidden_num = 128
num_classes=10

model_layers = [
    Conv2D(32, (3, 3), activation='relu', strides=(1,1), padding='same', input_shape=(32, 32, 3)),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu, depth_multiplier=3),
#     MaxPooling2D(2, 2),
    Dropout(rate =0.1),
    
    
    Conv2D(64, (3, 3), activation='relu', strides=(2, 2), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu),
#     MaxPooling2D(2, 2),
    Dropout(rate = 0.1),
    
    Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu),
#     MaxPooling2D(2, 2),
    Dropout(rate = 0.4),
    
    Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(1,1), strides=(1, 1), padding='same', activation=keras.activations.relu),
    
    
    Conv2D(256, (3, 3), activation='relu', strides=(2, 2), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu),
    
    
    
    Conv2D(512, (1, 1), activation='relu', strides=(2, 2), padding='same'),
    BatchNormalization(),
    DepthwiseConv2D(kernel_size=(1,1), strides=(1, 1), padding='same', activation=keras.activations.relu),
    Dropout(rate =0.3),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
] 
model_ini = Sequential(model_layers)

print(model_ini.summary())

checkpoint = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/cnn trust/update/parameters/model_CIFAR10_maxpool.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='max')  
model_ini.compile(optimizer='adam',
                 loss=tf.keras.losses.sparse_categorical_crossentropy, 
                 metrics=['accuracy'],run_eagerly=True)

history = model_ini.fit(train_images, train_label, batch_size=128, epochs=30, verbose=1, validation_data = (test_images, test_label), callbacks = [checkpoint])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import sklearn
from sklearn.metrics import auc, plot_precision_recall_curve

model_maxpool = tf.keras.models.load_model('/content/drive/MyDrive/cnn trust/update/parameters/model_CIFAR10_maxpool.h5')
model_maxpool.evaluate(test_images, test_label)

y_pred = model_maxpool.predict(test_images)
m = tf.keras.metrics.AUC(num_thresholds=200)
m.update_state(tf.one_hot(test_label[:,0],depth=10), y_pred)
auroc = m.result().numpy()

y_pred = tf.math.argmax(y_pred,axis=1)
precision, recall, f1, _ = precision_recall_fscore_support(test_label, y_pred,average='weighted')

precision, recall, f1, auroc

313/313 [==============================] - 2s 4ms/step - loss: 0.6178 - accuracy: 0.8269


(0.8328486137695199, 0.8269, 0.8270987799956391, 0.9749063)